# Accuracy VS Speed

When building a deep learning model for computer vision, there are many parameters that we may want to tune in order get the kind of model we need. Sometimes, we want models that perform at the highest possible accuracy it can achieve. Other times, we want models that can be packed into small machines to optimize for portability. By configuring these parameters, we can create models that are composed precisely to our needs.

## High accuracy models vs High speed models

While not always the case, high-speed models (smaller models and with faster the inference times) tend to be less performant than high-accuracy models (large complex models with slower inference times). 

As practitioners of computer vision, we want to be able to configure the trade-off between inference speeds and model performance so that we have better control over our models in different scenarios - especially when it comes to deploying our models into production pipelines. 

High accuracy models are most suited for scenarios where you're inferencing on a beefy server where resources are a problem. Sometimes these servers may live in the cloud, such as on Azure, while other times these servers may be physical boxes that are deployed at the site of production.

In contrast, high speed models (which tend to have a smaller memory footprint) are most commonly used in IoT scenarios or scenarios where we want to optimize for through-put.

This notebook will explore this trade-off and help you come of with the optimal model for your scenario.


---

Check out fastai version.

In [1]:
import fastai
fastai.__version__

'1.0.47'

Ensure edits to libraries are loaded and plotting is shown in the notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## High Accuracy Models

Lets start by looking at high-accuracy models. (In the meantime, don't worry about the model's speed or memory footprint.)

As you walked through [the previous notebook](01_training_introduction.ipynb), you may have wondered why we chose specific parameters for the model. These parameters include:
- model architecture (resnet50, resnet18, etc)
- learning rate
- number of epochs
- image sizes

It turns out, many of these parameters were chosen as the default parameters to optimize for high-accuracy models because of empirical findings.  

---

import sys
sys.path.append("../")
from pathlib import Path
from utils_ic.datasets import Urls, unzip_url, data_path
from fastai.vision import *
from fastai.metrics import error_rate, accuracy